compared with sg_5.ipynb


iccv-model 3 version 3: pocca + new trick + no merging + 100 epoch + 16 patch_num + new_fps + rstj

In [1]:
import os
import torch

import sys
sys.path.append("..")

from models.iccv_model_3 import SimAttention_ICCV_3
from data.shapenet_loader import ShapeNetCLS_RSTJ

In [ ]:
from datetime import datetime

# set gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# hy-paras
BATCS_SIZE = 16
PATCH_NUM = 16
EPOCHS = 100

# data
root = r'/home/ies/qhuang/H_AttentionProject/'
dataset = ShapeNetCLS_RSTJ(root, 1024)
trainDataLoader = torch.utils.data.DataLoader(dataset,
        batch_size=BATCS_SIZE,
        shuffle=True,
        num_workers=8,
        pin_memory=True)

# set model
model = SimAttention_ICCV_3(patch_num=PATCH_NUM)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-5)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.3)

def train_1_epoch(model, optimizer, data_loader, device):
        model.train()
        mean_loss = torch.zeros(1).to(device)
        optimizer.zero_grad()
        for step, data in enumerate(data_loader):
                aug1, aug2 = data
                # loss = model(aug1.to(device), aug2.to(device))
                loss = model(aug1.to(device=device, dtype=torch.float),
                             aug2.to(device=device, dtype=torch.float))
                loss = loss.mean()
                loss.backward()
                mean_loss = (mean_loss * step + loss.detach()) / (step + 1)  # update mean losses
                optimizer.step()
                optimizer.zero_grad()
        return mean_loss.item()

print('Start training at {}'.format(datetime.now()))
for epoch in range(0, EPOCHS):
    print('--------Epoch {} is running--------'.format(epoch))
    loss = train_1_epoch(model, optimizer, trainDataLoader, device)
    print("Loss: ", loss)
    print('\n')
    scheduler.step()
    if epoch%5==0 or (epoch+1)==EPOCHS:
        weight_name = 'cls_dg_iccv3_sg6_100_' + str(epoch) + '.pth'
        weight_path = os.path.join(r'/home/ies/qhuang/H_AttentionProject/weights/sg_6', weight_name)
        torch.save(model.state_dict(), weight_path)
        print('Model Saved!')

print("End at {}".format(datetime.now()))

Start training at 2023-04-21 03:42:29.862138
--------Epoch 0 is running--------
Loss:  0.4716184139251709


Model Saved!
--------Epoch 1 is running--------
